In [1]:
import numpy as np,numpy
import cv2
import sys
import math
import os#获取当前路径
from PIL import Image

In [56]:
# 从text文本中读取特征点
def readPoints(path) :
    # Create an array of points.
    points = [];
    # Read points
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))
    return points

#生成灰度图片并保存
def gray_img(img_path,save_path,save_name):
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    cv2.imwrite(save_path+save_name, img,[int(cv2.IMWRITE_PNG_COMPRESSION), 0])
    
#通过PIL包旋转图片显示并保存，输入为角度
def rotate_img(img_path,angle,save_path):
    im=Image.open(img_path)
    im_rotate=im.rotate(angle)
    im_rotate.show()
    im_rotate.save(save_path)
    
#求矩阵夹角,输出为角度而非弧度
def matrix_angle(point_path):
    #读取特征点
    points = readPoints(point_path)
    points = np.matrix(points)
    #计算重心点
    center=np.mean(points,axis=0)
    #计算特征点基于重心点的映射点
    pointsm=[]
    for i in range(len(points)):
        pointsm.append(((center[0,0]*2-points[i,0]),points[i,1]))
    pointsm=np.matrix(pointsm)
    #映射点和特征点求平均点
    avr_points=[]
    for i in range(len(points)):
        avr_points.append(((points[i,0]+pointsm[i,0])/2,(points[i,1]+pointsm[i,1])/2))
    avr_points=np.matrix(avr_points)
    avr_points=avr_points.astype(np.int)
    #计算特征点和平均特征点的均值和标准化
    aver_cols_p = np.mean(points,axis=0)
    stdx=np.std(points[:,0])
    stdy=np.std(points[:,1])
    stdn=((stdx)**2+(stdy)**2)**0.5
    s_points=np.matrix((points[:,:]-aver_cols_p)/stdn)#特征点标准化
    aver_cols_avr = np.mean(pointsm,axis=0)
    stdxm=np.std(avr_points[:,0])
    stdym=np.std(avr_points[:,1])
    stdnm=((stdxm)**2+(stdym)**2)**0.5
    s_points_avr=np.matrix((avr_points[:,:]-aver_cols_avr)/stdnm)#平均特征点标准化
    #特征点减去平均特征点求夹角
    angle=np.mean(np.mod(np.arctan(s_points[:,1]/s_points[:,0])-np.arctan(s_points_avr[:,1]/s_points_avr[:,0])+math.pi/2,math.pi)-math.pi/2)
    angle=angle/math.pi*180
    return(angle)

#根据夹角旋转特征点
def points_rotation(img_path,point_path,angle,save_path):
    #读取特征点并转换成矩阵
    points = readPoints(point_path) 
    points = np.matrix(points)#矩阵转换
    
    #获取图片宽度，高度
    im=Image.open(img_path)
    width,height = im.size
    #计算夹角的正弦余弦，输入要求是弧度
    sin=math.sin(angle*math.pi/180)
    cos=math.cos(angle*math.pi/180)
    #计算图片旋转中心点
    centerx=width/2
    centery=height/2
    #围绕图片中心点旋转坐标
    centery=height-centery
    x=points[:,0]
    y=points[:,1]
    y=height-y
    x1=(x-centerx)*cos-(y-centery)*sin+centerx
    y1=(x-centerx)*sin+(y-centery)*cos+centery
    y1=height-y1
    points=np.hstack((x1,y1))
    #存储到文档中
    np.savetxt(save_path+".txt",np.matrix(points),fmt='%i %i')

#根据特征点中心位置使用最小矩形截取并保存
def shape_face(file_path,point_path,save_path):
    img=cv2.imread(save_path)
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    black=np.array([0,0,0])
    mask = cv2.inRange(hsv,black,black,255)
    #腐蚀膨胀
    erode=cv2.erode(mask,None,iterations=1)
    dilate=cv2.dilate(erode,None,iterations=1)
    #将RGB图转化成二值图
    imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(imgray,10,255,cv2.THRESH_BINARY)
    contours = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cnt = contours[0]
    #boundingRect框图法
    x,y,w,h = cv2.boundingRect(cnt)
#     cv2.rectangle(img, (x, y), (x+w+10, y+h+10), (0, 0, 255), 3)
    img=img[y:y+h,x:x+w]                 
    cv2.imwrite(save_path,img)
    cv2.namedWindow("res",0)
    cv2.imshow('res',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#通过特征点的镜像对称来平均特征点
def average_points(points_path):
    #读取特征点
    points = read_points(point_path)
    points = np.matrix(points)
    #计算重心点
    center=np.mean(points,axis=0)
    #设定中间特征点，左侧特征点和右侧特征点
    index=[8,27,28,29,30,33,51,57,62,66]
    index_l=[ 0, 1, 2, 3, 4, 5, 6, 7,17,18,19,20,21,31,32,36,37,38,39,40,41,48,49,50,58,59,60,61,67]
    index_r=[16,15,14,13,12,11,10, 9,26,25,24,23,22,35,34,45,44,43,42,47,46,54,53,52,56,55,64,63,65]
    
    #提取中间特征点
    middle_points=np.delete(points,index_l+index_r,axis=0)
    #提取左右特征点
    left_points=[]
    right_points=[]
    for i in index_l:
        left_points.append((points[i,0],points[i,1]))
    left_points=np.matrix(left_points)
    for i in index_r:
        right_points.append((points[i,0],points[i,1]))
    right_points=np.matrix(right_points)
    
    #计算中轴特征点基于中心点的映射点
    middlem_points=[]
    for i in range(len(middle_points)):
        middlem_points.append(((center[0,0]*2-middle_points[i,0]),middle_points[i,1]))
    middlem_points=np.matrix(middlem_points)
    #计算左右特征点基于中心点的映射点
    leftm_points=[]
    rightm_points=[]
    for i in index_l:
        leftm_points.append(((center[0,0]*2-points[i,0]),points[i,1]))
    leftm_points=np.matrix(leftm_points)
    for i in index_r:
        rightm_points.append(((center[0,0]*2-points[i,0]),points[i,1]))
    rightm_points=np.matrix(rightm_points)
    #计算左右映射平均点
    avr_l=[]
    avr_r=[]
    avr_m=[]
    for i in range(len(left_points)):
        avr_l.append(((left_points[i,0]+rightm_points[i,0])/2,(left_points[i,1]+rightm_points[i,1])/2))
    avr_l=np.matrix(avr_l)
    for i in range(len(right_points)):
        avr_r.append(((right_points[i,0]+leftm_points[i,0])/2,(right_points[i,1]+leftm_points[i,1])/2))
    avr_r=np.matrix(avr_r)
    for i in range(len(middle_points)):
        avr_m.append(((middle_points[i,0]+middlem_points[i,0])/2,(middle_points[i,1]+middlem_points[i,1])/2))
    avr_m=np.matrix(avr_m)
    avr_points=np.vstack((avr_l,avr_r,avr_m))
    avr_points=avr_points.astype(np.int)
    return avr_points



In [57]:
cwd=os.getcwd()
if not os.path.exists(cwd+'/processedimg'):
    os.makedirs(cwd+'/processedimg')
name='m1.jpg'
save_path=cwd+'/processedimg/'+name
filename1 ='/home/fengchao/桌面/MonkeyFaceMerging/img/gray6.png'
filename2 ='/home/fengchao/桌面/MonkeyFaceMerging/img/h1.png'
points1 =cwd+'/points/gray6.png.txt'
points2 =cwd+'/points/h1.png.txt'
angle= matrix_angle(points1)
print(angle)
# rotate_img(filename1,angle,save_path)
points_rotation(filename1,points1,angle,save_path)

-3.47055126973
